# Develop & Curate – Einträge für Forschungsdatenrepositorien in re3data erstellen, bearbeiten, durchsuchen und nachnutzen
## 8. Bibliothekskongress 2022 Leipzig

### Anwendungsfall: Repositorien identifizieren, an deren Betrieb eine bestimmte Einrichtung beteiligt ist
Für einen Bericht sollen alle Forschungsdatenrepositorien identifiziert werden, an deren Betrieb das KIT beteiligt ist.
Repositorien, die diese Bedingung erfüllen, können über die re3data API ermittelt werden. Die API bietet auch die Möglichkeit, weitere Informationen über diese Repositorien abzurufen, wie der Name des Repositories oder eine Beschreibung.

#### Schritt 1: Pakete laden
Das Paket **httr** enthält die HTTP-Methode GET, mit der Daten von der re3data API angefordert werden. Die re3data API liefert Daten in XML. **xml2** enthält Funktionen für die Arbeit mit XML, z. B. zum Parsen oder Extrahieren des Inhalts bestimmter Elemente. Installieren Sie gegebenenfalls die Pakete, bevor Sie sie laden.

In [4]:
#install.packages("htttr")
#install.packages("xml2")
library(httr)
library(xml2)

Warning message:
"package 'httr' was built under R version 4.1.1"
Warning message:
"package 'xml2' was built under R version 4.1.2"


#### Schritt 2: Abfrageparameter definieren
Informationen über einzelne Repositorien können anhand der re3data ID extrahiert werden. Daher müssen zunächst re3data IDs von Repositories mit den gewünschten Eigenschaften identifiziert werden.

Die re3data API ermöglicht eine Suche in bestimmten Feldern über den endpoint **/api/beta/repositories**. Die Parameter, die über diesen endpoint durchsucht werden können, sind in der [Dokumentation der re3data API](https://www.re3data.org/api/doc) aufgeführt. Weitere Informationen zu re3data Metadaten, einschließlich Beschreibungen der verfügbaren Elemente und kontrollierten Vokabularen, finden Sie in der Dokumentation des [re3data Metadatenschemas](https://doi.org/10.2312/re3.006) (die API basiert auf Version 2.2 des re3data-Metadatenschemas).

Für den beschriebenen Anwendungsfall eignet sich am besten eine Suche nach der ROR ID des KIT (04t3en479) über alle Felder. Dafür wird an die base URL des API endpoints der string **query=04t3en479** angehängt.

ROR IDs von Einrichtungen können über die [ROR Suchfunktion](https://ror.org/search) ermittelt werden.

In [5]:
re3data_query <- "https://www.re3data.org/api/beta/repositories?query=04t3en479"

### Schritt 3: URLs für weitere Abfragen erstellen
Die im vorangegangenen Schritt definierte Abfrage kann über **GET** an die re3data API übergeben werden.

Die response wird mit **read_XML** geparst, da sie URLs für weitere Abfragen an den Endpunkt **/api/beta/repository** enthält. Diese URLs können mit einem einfachen XPath-Ausdruck identifiziert werden. Alle Elemente, die dem XPath-Ausdruck entsprechen, werden mit **xml_find_all** gefunden, und ihr Inhalt wird mit **xml_text** extrahiert.

Die drei Funktionen sind im folgenden Beispiel ineinander verschachtelt.

In [8]:
re3data_request <- GET(re3data_query) 

URLs <- xml_text(xml_find_all(read_xml(re3data_request), xpath = "//@href"))

#### Schritt 4: Definieren, welche Informationen über die Repositorien abgefragt werden sollen
Die im folgenden Codeblock definierte Funktion **extract_repository_info** extrahiert den Inhalt bestimmter XML-Elemente. Diese Funktion wird später verwendet, um die gewünschten Informationen aus der API response zu ziehen. Die Grundstruktur der Funktion ähnelt dem Prozess, der in Schritt 3 oben beschrieben wurde.
Die hier definierten XPath-Ausdrücke extrahieren die re3data ID, Namen, URL und Beschreibung für jedes Repositorium. Die Ergebnisse werden in einer Liste gespeichert, die später weiterverarbeitet werden kann.

Je nach Anwendungsfall kann diese Funktion angepasst werden, um andere Elemente und Attribute zu extrahieren. Einen Überblick über re3data Metadaten finden Sie in der Dokumentation des [re3data Metadatenschemas](https://doi.org/10.2312/re3.006) (die API verwendet Version 2.2 des re3data Metadatenschemas).

Die Funktion **xml_structure** aus dem Paket **xml2** kann sehr nützlich sein, um die Struktur von XML-Objekten zu analysieren und XPath-Ausdrücke zu definieren.

Bitte beachten Sie, dass die hier erwähnten Elemente in Version 2.2 des re3data Metadatenschemas für jedes Repositorium höchstens einmal vorkommen. Informationen zum Umgang mit Elementen, die mehrfach auftreten können, finden Sie in anderen Beispielen zur Nutzung der re3data API.

In [9]:
extract_repository_info <- function(repository_metadata_XML) {
  list(
    re3data_ID = xml_text(xml_find_all(repository_metadata_XML, "//r3d:re3data.orgIdentifier")),
    repositoryName = xml_text(xml_find_all(repository_metadata_XML, "//r3d:repositoryName")),
    repositoryUrl = xml_text(xml_find_all(repository_metadata_XML, "//r3d:repositoryURL")),
    description = xml_text(xml_find_all(repository_metadata_XML, "//r3d:description"))
  )
}

#### Schritt 5: Einen leeren Container für die Speicherung der Ergebnisse erstellen
**repository_info** ist ein leerer Container, in dem die Ergebnisse der API-Abfrage gespeichert werden können. Der data frame hat vier Spalten, die den in **extract_repository_info** definierten Metadatenelementen entsprechen.

In [10]:
repository_info <- data.frame(matrix(ncol = 4, nrow = 0))

colnames(repository_info) <- c("re3data_ID", "repositoryName", "repositoryUrl", "description")

#### Schritt 6: Gewünschte Informationen über Repositorien sammeln
Nach der Vorbereitung der Liste mit URLs, der Extraktionsfunktion und des leeren Containers für die Ergebnisse können diese Komponenten kombiniert werden. Der folgende Codeblock durchläuft die Liste der URLs mit einer for-Schleife. Für jedes Repositorium werden mit **GET** Daten von der re3data API angefordert. Die XML response wird mit **read_xml** geparst, bevor **extract_repository_info** aufgerufen wird. Die Ergebnisse werden dann als neue Zeile an **repository_info** angehängt.

In [11]:
for (url in URLs) {
  repository_metadata_request <- GET(url)
  repository_metadata_XML <-read_xml(repository_metadata_request) 
  results_list <- extract_repository_info(repository_metadata_XML)
  repository_info <- rbind(repository_info, results_list)
}

#### Ergebnisse
Die Ergebnisse wurden in **repository_info** gespeichert. Sie können mit **print** vollständig ausgegeben, mit **head** inspiziert oder mit **write.csv** lokal gespeichert werden.

In [13]:
print(repository_info)

     re3data_ID                                               repositoryName
1  r3d100012578                                                      KITopen
2  r3d100010087          Aktuelle Wetter- und Klimawerte deutscher Stationen
3  r3d100010407                                   Der Karlsruher Wolkenatlas
4  r3d100013265                                                    SupraBank
5  r3d100013303                                           Risklayer Explorer
6  r3d100013095          Informationssystem Graffiti in Deutschland (INGRID)
7  r3d100011896                                                      THEREDA
8  r3d100012861                          Virtuelles Skriptorium St. Matthias
9  r3d100010748                                                    chemotion
10 r3d100010406                                        Wetter, Wolken, Klima
11 r3d100012645                                                        HeRBi
12 r3d100013466                                                    RADAR4KIT